In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
import matplotlib
import tensorly as tl
import numpy as np
import json
import time
import random
from datetime import datetime
from tensorly.decomposition import non_negative_tucker
import collections
from pandas import Series
plt.rcParams['savefig.dpi'] = 300  # 图片像素
plt.rcParams['figure.dpi'] = 300  # 分辨率
plt.rcParams['figure.figsize'] = (20, 12)  # 尺寸
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False     # 正常显示负号

# 出租车GPS轨迹处理，保留car_id、经纬度、时间

In [2]:
#时间转秒
def time2miao(s):
    #print(s)
    shijian=s.split(' ')[1]
    h,m,s = shijian.split(":")
    return int(h) * 3600 + int(m) * 60 + int(s)
#出租车轨迹数据处理，保留需要的列。
def chuzucheguiji_chuli(chuzucheguiji_path,yueB_chepai_path,save_path):
    df_chuzucheguiji=pd.read_csv(open(chuzucheguiji_path))
    print(len(df_chuzucheguiji))
    df_yueB_chepai=pd.read_csv(open(yueB_chepai_path,encoding='utf-8'),dtype=str)
    df_chuzucheguiji_chuli=pd.merge(df_chuzucheguiji,df_yueB_chepai,left_on='车牌号码',right_on='chepai',how='inner')
    df_chuzucheguiji_chuli=df_chuzucheguiji_chuli[['car_id','经度','纬度','数据时间']] 
    df_chuzucheguiji_chuli=df_chuzucheguiji_chuli[df_chuzucheguiji_chuli['数据时间'].str.contains(' ')]
    df_chuzucheguiji_chuli['数据时间']=df_chuzucheguiji_chuli['数据时间'].apply(time2miao)
    print(len(df_chuzucheguiji_chuli))
    df_chuzucheguiji_chuli=df_chuzucheguiji_chuli.drop_duplicates()
    df_chuzucheguiji_chuli.to_csv(save_path,index=False)
    print(len(df_chuzucheguiji_chuli))
    print('##############')

In [3]:
'''
chuzucheguiji_chuli(r'F:\运营车识别\出租车GPS轨迹\20161205.csv',
                    r'C:\Users\Jeremy\Desktop\卡口\CarIdNumberInfo_yueB_UTF-8.csv',
                    r'F:\运营车识别\出租车GPS轨迹\20161205_chuli.csv')
chuzucheguiji_chuli(r'F:\运营车识别\出租车GPS轨迹\20161206.csv',
                    r'C:\Users\Jeremy\Desktop\卡口\CarIdNumberInfo_yueB_UTF-8.csv',
                    r'F:\运营车识别\出租车GPS轨迹\20161206_chuli.csv')
chuzucheguiji_chuli(r'F:\运营车识别\出租车GPS轨迹\20161207.csv',
                    r'C:\Users\Jeremy\Desktop\卡口\CarIdNumberInfo_yueB_UTF-8.csv',
                    r'F:\运营车识别\出租车GPS轨迹\20161207_chuli.csv')
chuzucheguiji_chuli(r'F:\运营车识别\出租车GPS轨迹\20161208.csv',
                    r'C:\Users\Jeremy\Desktop\卡口\CarIdNumberInfo_yueB_UTF-8.csv',
                    r'F:\运营车识别\出租车GPS轨迹\20161208_chuli.csv')
chuzucheguiji_chuli(r'F:\运营车识别\出租车GPS轨迹\20161209.csv',
                    r'C:\Users\Jeremy\Desktop\卡口\CarIdNumberInfo_yueB_UTF-8.csv',
                    r'F:\运营车识别\出租车GPS轨迹\20161209_chuli.csv')
chuzucheguiji_chuli(r'F:\运营车识别\出租车GPS轨迹\20161210.csv',
                    r'C:\Users\Jeremy\Desktop\卡口\CarIdNumberInfo_yueB_UTF-8.csv',
                    r'F:\运营车识别\出租车GPS轨迹\20161210_chuli.csv')
chuzucheguiji_chuli(r'F:\运营车识别\出租车GPS轨迹\20161211.csv',
                    r'C:\Users\Jeremy\Desktop\卡口\CarIdNumberInfo_yueB_UTF-8.csv',
                    r'F:\运营车识别\出租车GPS轨迹\20161211_chuli.csv')
'''

# 根据出租车卡口拍摄记录提取前后两个卡口通过的出租车id

In [4]:
#根据出租车卡口拍摄记录提取前后两个卡口通过的出租车id
def kakou_carid(chuzuche_path,save_path):
    chuzuche=pd.read_csv(open(chuzuche_path),index_col='car_id')
    chuzuche=chuzuche.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
    print(len(chuzuche))
    print(len(set(chuzuche.index)))
    kid_df=pd.DataFrame(columns=['q_kid','h_kid','car_id'])
    chushi_carid=-1
    q_kid=-1
    h_kid=-1
    #h=0 #计数
    for index, row in chuzuche.iterrows():
        #h=h+1
        #print(h)
        if index!=chushi_carid:
            chushi_carid=index
            q_kid=row['id']
        else:
            h_kid=row['id']
            new_line=pd.Series({'q_kid':q_kid,'h_kid':h_kid,'car_id':index})
            kid_df=kid_df.append(new_line,ignore_index=True)        
            q_kid=row['id']
    print(len(kid_df))#len(set(chuzuche.index)) + len(kid_df) = len(chuzuche)
    kid_df=kid_df.drop_duplicates(['q_kid','h_kid','car_id'])
    print(len(kid_df))
    kid_df=kid_df.sort_values(ascending=True,by=['q_kid','h_kid','car_id'])#从小到大排序
    kid_df.reset_index(drop=True,inplace=True)#重置索引
    
    
    kid_df_hebing=pd.DataFrame(columns=['q_kid','h_kid','car_id_list'])
    kid_df_duplicated=kid_df.duplicated(['q_kid','h_kid'])
    print(len(kid_df_duplicated)) #len(kid_df_duplicated) = len(kid_df)
    print(sum(kid_df_duplicated)) #sum(kid_df_duplicated) + len(kid_df_hebing) = len(kid_df)
    car_id_list=[]
    q_kid=-1
    h_kid=-1
    i=0#计数
    a=0#计数 
    for index, row in kid_df.iterrows():
        i=i+1
        #print(i)
        if kid_df_duplicated[index]==False:
            if car_id_list:
                #car_id_list不为空
                a=a+len(car_id_list)
                new_line=pd.Series({'q_kid':q_kid,'h_kid':h_kid,'car_id_list':car_id_list})
                kid_df_hebing=kid_df_hebing.append(new_line,ignore_index=True)
                car_id_list=[]
                q_kid=-1
                h_kid=-1
                car_id_list.append(row['car_id'])
                q_kid=row['q_kid']
                h_kid=row['h_kid']

            else:
                #car_id_list为空
                car_id_list.append(row['car_id'])
                q_kid=row['q_kid']
                h_kid=row['h_kid']

        else:
            car_id_list.append(row['car_id'])
        if i==len(kid_df):
            a=a+len(car_id_list)
            new_line=pd.Series({'q_kid':q_kid,'h_kid':h_kid,'car_id_list':car_id_list})
            kid_df_hebing=kid_df_hebing.append(new_line,ignore_index=True)
    print(len(kid_df_hebing))
    print(a) #a = len(kid_df)
    kid_df_hebing.to_csv(save_path,index=False)

In [5]:
'''
kakou_carid(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_chuzuche.csv',
           r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_chuzuche_kakou_carid.csv')
kakou_carid(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161206_title_chuzuche.csv',
           r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161206_title_chuzuche_kakou_carid.csv')
kakou_carid(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161207_title_chuzuche.csv',
           r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161207_title_chuzuche_kakou_carid.csv')
kakou_carid(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161208_title_chuzuche.csv',
           r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161208_title_chuzuche_kakou_carid.csv')
kakou_carid(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161209_title_chuzuche.csv',
           r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161209_title_chuzuche_kakou_carid.csv')
kakou_carid(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_chuzuche.csv',
           r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_chuzuche_kakou_carid.csv')
kakou_carid(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161211_title_chuzuche.csv',
           r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161211_title_chuzuche_kakou_carid.csv')
'''

# 计算非运营车通过的前后两个卡口间可能经过哪些卡口

In [6]:
#删除只有一次出行车辆
def shanchu_yicichuxing(df):
    #统计拍摄次数
    tongji=Series(collections.Counter(df.index)).to_frame().rename(columns={0:'num'})
    #拍摄次数大于1的车辆id
    carid=tongji[tongji['num']>1].index
    #保留拍摄次数大于1的车辆
    return df.loc[carid]

使用箱线图法切割一日出行

In [7]:
#每辆非运营车通过两卡口之间的时间差，需要运行10mins
def shijiancha(df):
    df=df.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
    #print(df)
    kakou_shijiancha=[]
    chushi_carid=-1
    chushi_time=-1
    #a=0#计数
    for index, row in df.iterrows():
        #print(a)
        #a=a+1
        if index!=chushi_carid:
            chushi_carid=index
            chushi_time=row['time']
        else:
            kakou_shijiancha.append(row['time']-chushi_time)
            chushi_time=row['time']
            #print(kakou_shijiancha)
    #print(kakou_shijiancha)
    return kakou_shijiancha

In [8]:
#计算上下四分位数,lis按从小到大排过序
def count_quartiles(lis):
    s=pd.Series(lis)
    q1=s.quantile(q=0.25)
    q3=s.quantile(q=0.75)
    return q1,q3

In [9]:
#计算上下边缘,lis按从小到大排过序
def count_margin(q1, q3, a):
    iqr = q3 - q1
    upper = q3 + a*iqr
    floor = q1 - a*iqr
    return upper, floor

In [10]:
#切割一日出行
def qiege(df, upper, floor):
    df=df.sort_values(ascending=True,by=['time'])#时间从小到大排序
    qiege_df=[] #df切割后放入qiege_df
    chushi_df=pd.DataFrame(columns=df.columns)
    chushi_df.index.name='car_id'
    chushi_time=-1
    i=0
    for index, row in df.iterrows():
        i=i+1
        if i==1:
            chushi_time=row['time']
            chushi_df=chushi_df.append(row)
        else:#
            if row['time']-chushi_time>=upper:
                qiege_df.append(chushi_df)
                chushi_df=pd.DataFrame(columns=df.columns)
                chushi_df.index.name='car_id'
                chushi_df=chushi_df.append(row)
            elif row['time']-chushi_time<=floor:
                chushi_df=chushi_df.append(row)
            else:
                chushi_df=chushi_df.append(row) 
            chushi_time=row['time']
        if i==len(df):
            qiege_df.append(chushi_df)
    return qiege_df

统计非运营车辆经过哪些前后卡口并保存

In [11]:
#统计df中车辆经过哪些前后卡口并保存
def tongji_feiyunying(df,upper, floor):#feiyunyingche_1205：df
    #统计星期一非运营车辆经过哪些路口
    df=df.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
    car_ids=list(set(df.index))
    q_h_kid=[]#存储星期一非运营车辆通过的前后路口
    n=0
    for car_id in car_ids:
        n=n+1
        #print(n)
        one_feiyunying=df.loc[car_id]
        qiege_dfs=qiege(one_feiyunying,upper, floor)
        for qiege_df in qiege_dfs:
            #print(qiege_df)
            k_ids=list(qiege_df['id'])
            #q_kid=qiege_df['id'].iloc[0]
            #print(k_ids)
            for index in range(len(k_ids)-1):
                q_h=[k_ids[index],k_ids[index+1]]
                if q_h not in q_h_kid:
                    q_h_kid.append(q_h)
                #print(a)

                #print(q_h_kid)
    df_q_h_kid=pd.DataFrame(q_h_kid,columns=['q_id','h_id','time'])
    return df_q_h_kid

In [12]:
feiyunyingche_1205=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1205=shanchu_yicichuxing(feiyunyingche_1205)
list_kakou_shijiancha=shijiancha(feiyunyingche_1205)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
#print(len(list_kakou_shijiancha))
#print(q1, q3)
#print(upper, floor)
tongji_feiyunying(feiyunyingche_1205,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_feiyunying_q_h_kid.csv',index=False)

D:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [13]:
feiyunyingche_1210=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_feiyunyingche_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1210=shanchu_yicichuxing(feiyunyingche_1210)
list_kakou_shijiancha=shijiancha(feiyunyingche_1210)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
#print(len(list_kakou_shijiancha))
#print(q1, q3)
#print(upper, floor)
tongji_feiyunying(feiyunyingche_1210,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_feiyunying_q_h_kid.csv',index=False)

D:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
'''
df_chuzuche_05=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_chuzuche_zuobiao.csv'),index_col='car_id')
df_chuzuche_05=shanchu_yicichuxing(df_chuzuche_05)
df_chuzuche_05=df_chuzuche_05.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序

df_chuzuche_06=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161206_title_chuzuche_zuobiao.csv'),index_col='car_id')
df_chuzuche_06=shanchu_yicichuxing(df_chuzuche_06)
df_chuzuche_06=df_chuzuche_06.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序

df_chuzuche_07=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161207_title_chuzuche_zuobiao.csv'),index_col='car_id')
df_chuzuche_07=shanchu_yicichuxing(df_chuzuche_07)
df_chuzuche_07=df_chuzuche_07.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序

df_chuzuche_08=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161208_title_chuzuche_zuobiao.csv'),index_col='car_id')
df_chuzuche_08=shanchu_yicichuxing(df_chuzuche_08)
df_chuzuche_08=df_chuzuche_08.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序

df_chuzuche_09=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161209_title_chuzuche_zuobiao.csv'),index_col='car_id')
df_chuzuche_09=shanchu_yicichuxing(df_chuzuche_09)
df_chuzuche_09=df_chuzuche_09.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序

df_chuzuche_10=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_chuzuche_zuobiao.csv'),index_col='car_id')
df_chuzuche_10=shanchu_yicichuxing(df_chuzuche_10)
df_chuzuche_10=df_chuzuche_10.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序

df_chuzuche_11=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161211_title_chuzuche_zuobiao.csv'),index_col='car_id')
df_chuzuche_11=shanchu_yicichuxing(df_chuzuche_11)
df_chuzuche_11=df_chuzuche_11.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
'''

In [ ]:
'''
#按出租车id切割df
def qiege_chuzuche(df):
    qiege_df=[] #df切割后放入qiege_df
    car_ids=list(set(df.index))
    for car_id in car_ids:
        qiege_df.append(df.loc[car_id])
    return qiege_df
'''

In [ ]:
'''
#运行约10mins
#计算出租车通过卡口间流量
def kakou_liuliang(datas):
    q_h_kid=[]#存储出租车通过的前后路口
    all_kid=[]
    for data in datas:
        all_kid=all_kid+list(data['id'])
        list_chuzuche_qiege=qiege_chuzuche(data)
        for one_chuzuche in list_chuzuche_qiege:
            #print(len(one_chuzuche))
            k_ids=list(one_chuzuche['id'])
            #print(len(k_ids))
            for index in range(len(k_ids)-1):
                q_h=[k_ids[index],k_ids[index+1]]
                q_h_kid.append(q_h)
    all_kid=list(set(all_kid))
    df_kakou_liuliang=pd.DataFrame(index=all_kid,columns=all_kid).fillna(0)#index表示前卡口，column表示后卡口 
    for q_h in q_h_kid:
        q=q_h[0]
        h=q_h[1]
        #print(q)
        #print(h)
        df_kakou_liuliang.loc[q,h]=df_kakou_liuliang.loc[q,h]+1
        #print(test1)
    return df_kakou_liuliang
'''

In [ ]:
'''
#df_kakou_liuliang有41409个大于0的单元 ,总流量和为3281767
df_kakou_liuliang=kakou_liuliang([df_chuzuche_05,df_chuzuche_06,df_chuzuche_07,df_chuzuche_08,df_chuzuche_09,df_chuzuche_10,df_chuzuche_11])
df_kakou_liuliang.to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_liuliang.csv')
sum(df_kakou_liuliang.sum())
'''

In [ ]:
'''
#根据前后卡口id，输出预测经过的卡口路径
def q_h_kakou(q,h,datas):
    q_h_guiji = pd.DataFrame(columns=['q_kid','h_kid','kakou_guiji','car_id','time','day']) 
    day=0
    for data in datas:
        data['car_id']=data.index
        data=data.reset_index(drop=True)
        day=day+1
        #print(day)
        
        list_chuzuche_qiege=[] #data切割后放入list_chuzuche_qiege
        car_ids=list(set(data['car_id']))
        for car_id in car_ids:
            list_chuzuche_qiege.append(data[data['car_id']==car_id])
            
        for one_chuzuche in list_chuzuche_qiege:
            #print('a')
            if (q in list(one_chuzuche['id'])) & (h in list(one_chuzuche['id'])):
                #print('b')
                df_q=one_chuzuche[one_chuzuche['id']==q]
                df_h=one_chuzuche[one_chuzuche['id']==h]
                df_q_h=df_q.append(df_h)
                df_q_h=df_q_h.sort_values(ascending=True,by=['time'])#时间从小到大排序
                q_index=-1
                h_index=-1
                list_index=[]
                #print(df_q_h)
                for index in range(len(df_q_h)):#m.iloc[1:2]['id'].iloc[0]
                    #print(index)
                    #print(df_q_h.iloc[index:index+1]['id'].iloc[0])
                    if df_q_h.iloc[index:index+1]['id'].iloc[0]==q:
                        q_index=df_q_h.iloc[index:index+1].index[0]#记录索引 
                    if (df_q_h.iloc[index:index+1]['id'].iloc[0]==h) & (q_index!=-1):       
                        h_index=df_q_h.iloc[index:index+1].index[0]
                        list_index.append([q_index,h_index])
                        q_index=-1
                        h_index=-1
                #print(list_index)
                for i in list_index:
                    df_kakou_guiji=one_chuzuche.loc[i[0]+1:i[1]]
                    time=df_kakou_guiji['time'][-1:].iloc[0]-df_kakou_guiji['time'][:1].iloc[0]
                    kakou_guiji=''
                    for k in df_kakou_guiji['id']:
                        #print(k)
                        #print(type(k))
                        #print(''+'?')
                        #print(''+'?'+str(k))
                        kakou_guiji=kakou_guiji+'?'+str(k)
                    kakou_guiji=kakou_guiji[1:]
                    new_line=pd.Series({'q_kid':q,'h_kid':h,'kakou_guiji':kakou_guiji,'car_id':one_chuzuche['car_id'].iloc[0],'time':time,'day':str(day)})          
                    q_h_guiji=q_h_guiji.append(new_line,ignore_index=True) 
    q_h_guiji['time']=q_h_guiji['time'].astype(float)
    q_h_guiji_groupby_time=pd.DataFrame(q_h_guiji['time'].groupby(q_h_guiji['kakou_guiji']).mean())
    q_h_guiji_groupby_time['kakou_guiji']=q_h_guiji_groupby_time.index
    q_h_guiji_groupby_time=q_h_guiji_groupby_time.reset_index(drop=True)
    q_h_guiji=pd.merge(q_h_guiji.drop_duplicates(['kakou_guiji']),q_h_guiji_groupby_time,on='kakou_guiji',how='inner')
    q_h_guiji=q_h_guiji.drop(['time_x'], axis=1)
    q_h_guiji.rename(columns={'time_y':'time'}, inplace = True)
    return q_h_guiji
'''

In [ ]:
#test=q_h_kakou('10300202','10103502',[df_chuzuche_05,df_chuzuche_10])

In [ ]:
'''
#计算时间相似性，并归一化
def time_consistency(t_virtual,list_t_average):
    gailve=[]
    for t_average in list_t_average:
        if t_average<=2*t_virtual and t_average>=t_virtual:
            gailve.append(2-t_average/t_virtual) 
        elif t_average<t_virtual:
            gailve.append(t_average/t_virtual) 
        else:
            gailve.append(0.01) 
    gailve=norm(gailve)
    return gailve
'''

In [ ]:
'''
#归一化
def norm(l):
    l=np.array(l)
    return l/sum(l)
'''

In [ ]:
'''
#从l中按概率p采样
def sample(l,p):
    return np.random.choice(a=l, size=1, p=p)[0]
'''

In [ ]:
'''
#根据qiege_df中的卡口，补全可能经过的卡口，未完成
def a(qiege_df):
    if(len(qiege_df))>1:
        qiege_df=qiege_df.sort_values(ascending=True,by=['time'])#时间从小到大排序
        for i in range(len(qiege_df)-1):
            time_cha=qiege_df.iloc[i+1]['time']-qiege_df.iloc[i]['time']
            q_h_guiji=q_h_kakou(qiege_df.iloc[i]['id'],qiege_df.iloc[i+1]['id'],[df_chuzuche_05,df_chuzuche_10])
            #print(len(q_h_guiji))
            q_h_guiji['gailve']=time_consistency(time_cha,list(q_h_guiji['time']))
            #print(q_h_guiji['gailve'])
            buquan_guiji=q_h_guiji.loc[sample(list(q_h_guiji.index),list(q_h_guiji['gailve']))]
            #print(buquan_guiji)
            #print(buquan_guiji['kakou_guiji'])
        return
    else:
        return
'''

In [ ]:
'''
feiyunyingche_1205=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_zuobiao.csv'),index_col='car_id')
feiyunyingche_1205=shanchu_yicichuxing(feiyunyingche_1205)
feiyunyingche_1205=feiyunyingche_1205.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
list_kakou_shijiancha=shijiancha(feiyunyingche_1205)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
car_ids=list(set(feiyunyingche_1205.index))
for car_id in car_ids:
    one_feiyunying=df.loc[car_id]
    qiege_dfs=qiege(one_feiyunying,upper, floor)
    for qiege_df in qiege_dfs:
        a(qiege_df)
'''

# 从qiege_df提取前后两个卡口的id，从kakou_carid_path随机提取通过两个卡口id的出租车（有时间限制，未使用粒子滤波），作为轨迹。

In [22]:
#选一辆非运营车的卡口轨迹做测试
feiyunyingche_1205=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1205=shanchu_yicichuxing(feiyunyingche_1205)
feiyunyingche_1205=feiyunyingche_1205.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
list_kakou_shijiancha=shijiancha(feiyunyingche_1205)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
#选一辆非运营车的卡口轨迹做测试
test_feiyunyingche_1205_carid=23068720
test_feiyunyingche_1205=feiyunyingche_1205.loc[test_feiyunyingche_1205_carid]
#test_feiyunyingche_1205
qiege_df=qiege(test_feiyunyingche_1205,upper, floor)
for i in qiege_df:
    print(i)
print(len(qiege_df))

                id type   time          x_m          y_m
car_id                                                  
23068720  30701902    3  25418  490384.3695  2516065.026
23068720  20504603    3  27295  498452.8076  2508100.642
23068720  20504601    3  27609  501332.5581  2502798.101
23068720  20104607    3  28729  501520.0665  2498730.692
23068720  10100206    3  29001  504673.5631  2495890.231
23068720  20104605    3  34550  501735.4416  2496519.496
23068720  20504602    3  34890  501323.7116  2502858.452
23068720  20504604    3  35195  498439.6582  2508150.033
23068720  30707901    3  40544  490501.7916  2519054.732
                id type   time          x_m          y_m
car_id                                                  
23068720  20504603    3  61649  498452.8076  2508100.642
23068720  20504601    3  61958  501332.5581  2502798.101
23068720  20104607    3  62463  501520.0665  2498730.692
23068720  10100204    3  63015  507263.8896  2495966.885
23068720  10200104    3  63284 

In [157]:
def tiqu_guiji(qiege_df,kakou_carid_path,chuzuche_path,chuzucheguiji_path,new_kakou_location_m): 
    df_kakou_carid=pd.read_csv(open(kakou_carid_path))
    df_chuzuche=pd.read_csv(open(chuzuche_path))
    df_chuzucheguiji=pd.read_csv(open(chuzucheguiji_path,encoding='utf-8'))
    save_df=pd.DataFrame(columns=['lujing_id','car_id','经度','纬度'])
    #print(df_kakou_carid)
    #print(df_chuzuche)
    a=0
    for d in qiege_df:
        a=a+1
        q_kid=-1
        h_kid=-1
        i=0
        for index, row in d.iterrows():
            i=i+1
            if i == 1 :
                q_kid=row['id']
            else:
                h_kid=row['id']
                car_id_list=df_kakou_carid[(df_kakou_carid['q_kid']==q_kid) & (df_kakou_carid['h_kid']==h_kid)].reset_index(drop=True).at[0,'car_id_list']  
                #print(car_id_list)
                car_id_list=json.loads(car_id_list)
                #print(car_id_list)
                df_chuzuche_ziji=df_chuzuche[df_chuzuche['car_id'].isin(car_id_list)]         
                df_chuzuche_ziji=df_chuzuche_ziji.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
                df_chuzuche_ziji.reset_index(drop=True,inplace=True)#重置索引
                #print(df_chuzuche_ziji)
                df_chuzucheguiji_ziji_list=[]
                m=0
                for index_j, row_j in df_chuzuche_ziji.iterrows():
                    m=m+1
                    if m!=len(df_chuzuche_ziji):
                        #print(index_j)
                        #print(row_j)
                        #print(len(df_chuzuche_ziji))
                        #print(df_chuzuche_ziji.iloc[index_j+1]['id'])
                        if row_j['id']==q_kid and df_chuzuche_ziji.iloc[index_j+1]['id']==h_kid and row_j['car_id']==df_chuzuche_ziji.iloc[index_j+1]['car_id']:  
                            #print(row_j['time'])
                            #print(df_chuzuche_ziji.ix[index_j+1,'time'])
                            #print(row_j['car_id'])
                            df_chuzucheguiji_ziji=df_chuzucheguiji[df_chuzucheguiji['car_id']==row_j['car_id']]
                            df_chuzucheguiji_ziji=df_chuzucheguiji_ziji[(df_chuzucheguiji_ziji['数据时间']>row_j['time']) & (df_chuzucheguiji_ziji['数据时间']<df_chuzuche_ziji.iloc[index_j+1]['time'])]
                            df_chuzucheguiji_ziji=df_chuzucheguiji_ziji.sort_values(ascending=True,by=['数据时间'])#时间从小到大排序
                            if df_chuzucheguiji_ziji.empty!=True:#df_chuzucheguiji_ziji不为空执行if
                                df_chuzucheguiji_ziji_list.append(df_chuzucheguiji_ziji)
                #从df_chuzucheguiji_ziji_list中随机抽取一个
                #print('############################')
                random.seed(10)
                #print(len(df_chuzucheguiji_ziji_list))
                #print(df_chuzucheguiji_ziji_list)
                random.shuffle(df_chuzucheguiji_ziji_list)#打乱df_chuzucheguiji_ziji_list顺序
                #print(df_chuzucheguiji_ziji_list)
                df_chuzucheguiji_ziji = random.sample(df_chuzucheguiji_ziji_list, 1)[0]
                #print(df_chuzucheguiji_ziji)
                df_chuzucheguiji_ziji['lujing_id']=[a]*len(df_chuzucheguiji_ziji)
                #print(df_chuzucheguiji_ziji)
                df_chuzucheguiji_ziji=df_chuzucheguiji_ziji.drop(['数据时间'], axis=1)
                ##############
                new_line=pd.Series({'lujing_id':a,'car_id':index,'经度':new_kakou_location_m[new_kakou_location_m['k_id']==q_kid].reset_index(drop=True).at[0,'x']  ,'纬度':new_kakou_location_m[new_kakou_location_m['k_id']==q_kid].reset_index(drop=True).at[0,'y']})
                save_df=save_df.append(new_line,ignore_index=True) 
                save_df=save_df.append(df_chuzucheguiji_ziji,ignore_index=True)
                ##############
                new_line=pd.Series({'lujing_id':a,'car_id':index,'经度':new_kakou_location_m[new_kakou_location_m['k_id']==h_kid].reset_index(drop=True).at[0,'x']  ,'纬度':new_kakou_location_m[new_kakou_location_m['k_id']==h_kid].reset_index(drop=True).at[0,'y']})
                save_df=save_df.append(new_line,ignore_index=True) 
                #print(df_chuzuche_ziji)
                q_kid=row['id']
    #删除save_df中经纬度连续重复值
    save_df=save_df[save_df[['经度','纬度']].diff().fillna(1).abs().sum(axis=1)!=0]
    save_df.reset_index(drop=True,inplace=True)#重置索引
    #删除save_df中经纬度变化较小值
    save_df=save_df[save_df[['经度','纬度']].diff().fillna(1).abs().sum(axis=1)>0.00025]
    save_df.reset_index(drop=True,inplace=True)#重置索引
    #print(save_df)
    return save_df
    #save_df.to_csv(save_path,index=False)#一辆车的轨迹还原

In [159]:
new_kakou_location_m=pd.read_csv(open(r'F:\运营车识别\new_kakou_location_m.csv'))
test_car_guiji_weipipei=tiqu_guiji(qiege_df,
          r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_chuzuche_kakou_carid.csv',   
          r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_chuzuche.csv',
          r'F:\运营车识别\出租车GPS轨迹\20161205_chuli.csv',new_kakou_location_m
          )#一辆车的轨迹还原

In [54]:
test_car_guiji_weipipei['car_id']=[test_feiyunyingche_1205_carid]*len(test_car_guiji_weipipei)
test_car_guiji_weipipei.to_csv(r'C:\cygwin64\home\Jeremy\fmm\example\wyk_guiji_pipei\test_car_guiji_weipipei.csv',index=False)
test_car_guiji_weipipei

,lujing_id,car_id,经度,纬度
0,1.0,23068720,113.906386,22.742866
1,1.0,23068720,113.906531,22.740558
2,1.0,23068720,113.906913,22.738422
3,1.0,23068720,113.906982,22.737373
4,1.0,23068720,113.907248,22.731306
...,...,...,...,...
183,3.0,23068720,113.995430,22.656060
184,3.0,23068720,113.993625,22.657905
185,3.0,23068720,113.991553,22.659599
186,3.0,23068720,113.989771,22.661078


In [55]:
test_car_kakou_yipipei=pd.DataFrame(columns=test_feiyunyingche_1205.columns)
test_car_kakou_yipipei['lujing_id']=''
i=0
for d in qiege_df:
    i=i+1
    d['lujing_id']=[i]*len(d)
    test_car_kakou_yipipei=test_car_kakou_yipipei.append(d)
test_car_kakou_yipipei.index.name='car_id'

test_car_kakou_yipipei_merge=pd.merge(test_car_kakou_yipipei,new_kakou_location_m,left_on='id',right_on='k_id',how='inner')
test_car_kakou_yipipei_merge.index=test_car_kakou_yipipei.index
test_car_kakou_yipipei_merge=test_car_kakou_yipipei_merge.sort_values(ascending=True,by=['time'])#时间从小到大排序
test_car_kakou_yipipei=test_car_kakou_yipipei_merge.drop(['x_m_y', 'y_m_y','id'], axis=1)
test_car_kakou_yipipei.rename(columns={'x_m_x':'x_m', 'y_m_x':'y_m', 'x':'经度', 'y':'纬度'}, inplace = True)
test_car_kakou_yipipei.to_csv(r'C:\cygwin64\home\Jeremy\fmm\example\wyk_guiji_pipei\test_car_kakou_yipipei.csv')
test_car_kakou_yipipei#一辆车的卡口轨迹

,type,time,x_m,y_m,lujing_id,k_id,经度,纬度
car_id,,,,,,,,
23068720,3,25418,490384.3695,2516065.026,1,30701902,113.906386,22.742866
23068720,3,27295,498452.8076,2508100.642,1,20504603,113.984945,22.670973
23068720,3,27609,501332.5581,2502798.101,1,20504601,114.012962,22.623090
23068720,3,28729,501520.0665,2498730.692,1,20104607,114.014782,22.586360
23068720,3,29001,504673.5631,2495890.231,1,10100206,114.045440,22.560704
23068720,3,34550,501735.4416,2496519.496,1,20104605,114.016874,22.566392
23068720,3,34890,501323.7116,2502858.452,1,20504602,114.012876,22.623635
23068720,3,35195,498439.6582,2508150.033,1,20504604,113.984817,22.671419
23068720,3,40544,490501.7916,2519054.732,1,30707901,113.907511,22.769864


In [12]:
#def path_link_ﬂow(l):
#    return l.min()